In [1]:
%matplotlib inline

import librosa
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import torch
import os
import sys
import glob
import pickle
import copy
import random
import time
import traceback
import shutil

from multiprocessing import Process, Manager

# Config

In [2]:
PROJECT_ROOT = '/workspace/GREAT_ASV_system/'
sys.path.append(PROJECT_ROOT)

In [3]:
OPT_INDEX = '/workspace/DATASET/server9/STD_SRE_EGS'
if not os.path.exists(OPT_INDEX):
    os.mkdir(OPT_INDEX)

In [4]:
SRE_TRAIN_DIR = '/workspace/DATASET/server9/SRE_SWBD_WAV'
AUG_INFO_DIR = '/workspace/DATASET/server9/STD_musan&rir_info'

In [5]:
INFO_DIR = os.path.join(OPT_INDEX, 'STD_INFO')
if not os.path.exists(INFO_DIR):
    os.mkdir(INFO_DIR)

spk2id = os.path.join(INFO_DIR, 'spk2id')
spk2utt_dict = os.path.join(INFO_DIR, 'spk2utt_dict')
spk2utt_train_dict = os.path.join(INFO_DIR, 'spk2utt_train_dict')
spk2utt_val_dict = os.path.join(INFO_DIR, 'spk2utt_val_dict')
spk2utt_train_len = os.path.join(INFO_DIR, 'spk2utt_train_len')
spk2utt_val_len = os.path.join(INFO_DIR, 'spk2utt_val_len')

In [6]:
val_utt_num = 200

# Make spk2utt dict

In [7]:
spk2utt = {}
spk2utt_train = {}
spk2utt_val = {}

with open(spk2id, 'w') as f:
    pass

label_dir = glob.glob(SRE_TRAIN_DIR+'/*/*')
label = [i.split('/')[-1] for i in label_dir]

for count, i in enumerate(label):
    with open(spk2id, 'a') as f:
        f.write(str(count)+','+i+'\n')
    spk2utt[count] = glob.glob(os.path.join(SRE_TRAIN_DIR, 'SRE16_TRAIN', i)+'/*.wav')

In [8]:
len(spk2utt)

7001

In [9]:
count = 0
count_list = []
for i in spk2utt:
    count += len(spk2utt[i])
    count_list.append(len(spk2utt[i]))
print(count)

91261


In [10]:
with open(spk2utt_dict, 'wb') as handle:
    pickle.dump(spk2utt, handle)

with open(spk2utt_dict, 'rb') as handle:
    spk2utt = pickle.load(handle)

In [11]:
utt_length = []
for i in spk2utt:
    utt_length.append(len(spk2utt[i]))

In [12]:
utt_length = np.argsort(utt_length)[::-1]

In [13]:
spk2utt_train = copy.deepcopy(spk2utt)
for i in range(val_utt_num):   
    spk2utt_val[utt_length[i]] = [spk2utt[utt_length[i]][0]]
    spk2utt_train[utt_length[i]] = spk2utt[utt_length[i]][1:]

In [14]:
count = 0
count_list = []
for i in spk2utt_train:
    count += len(spk2utt_train[i])
    count_list.append(len(spk2utt[i]))
print(count)

91061


In [15]:
with open(spk2utt_train_dict, 'wb') as handle:
    pickle.dump(spk2utt_train, handle)

In [16]:
count = 0
count_list = []
for i in spk2utt_val:
    count += len(spk2utt_val[i])
    count_list.append(len(spk2utt[i]))
print(count)

200


In [17]:
with open(spk2utt_val_dict, 'wb') as handle:
    pickle.dump(spk2utt_val, handle)

## Use multi-process to check read wav

In [18]:
manager = Manager()
spk2utt_train_len_dict_m = manager.list()
eer_m = manager.list()

In [19]:
# err = []
with open(spk2utt_train_dict, 'rb') as handle:
    spk2utt_train = pickle.load(handle)

In [20]:
spk2utt_train_splitlist = []
num_per_process = len(spk2utt_train) // 10
for i in range(9):
    spk2utt_train_splitlist.append([j for j in range(i*num_per_process, (i+1)*num_per_process)])
    spk2utt_train_len_dict_m.append({})
spk2utt_train_splitlist.append([j for j in range(9*num_per_process, len(spk2utt_train))])
spk2utt_train_len_dict_m.append({})

In [21]:
def read_wav_m(p, spk2utt_train, spk2utt_train_split, spk2utt_train_len_dict_m, eer_m):
    spk2utt_train_len_dict = {}
    for count, i in enumerate(spk2utt_train_split):
        for utt in spk2utt_train[i]:
            try:
                spk2utt_train_len_dict[utt] = librosa.core.load(utt, sr=16000)[0].shape[0]
            except Exception:
                eer_m.append([p, utt])
                continue
        print(count)
    spk2utt_train_len_dict_m[p] = spk2utt_train_len_dict

In [ ]:
with open(spk2utt_train_dict, 'rb') as handle:
    spk2utt_train = pickle.load(handle)

processes = [Process(target = read_wav_m, args = (i, spk2utt_train, spk2utt_train_splitlist[i], spk2utt_train_len_dict_m, eer_m)) for i in range(10)]
[p.start() for p in processes]
joined = [p.join() for p in processes]

0
0
1
0
0
0
0
1
1
0
0
1
1
0
2
2
1
2
2
2
1
1
3
4
2
3
1
3
3
4
2
4
3
5
6
5
6
7
4
3
2
3
4
4
5
5
4
5
2
6
6
5
7
5
8
6
3
0
1
7
6
6
3
7
4
9
2
3
4
7
5
8
10
5
6
4
6
7
7
5
7
6
8
11
12
8
9
8
9
9
13
8
14
10
9
7
8
15
9
10
7
11
9
8
10
11
12
12
8
9
10
8
13
10
11
11
9
12
14
13
15
12
10
14
9
11
12
10
16
11
10
10
13
11
12
14
11
12
17
11
15
13
16
12
17
13
14
15
13
14
18
12
15
19
13
20
14
13
15
16
16
16
14
16
17
18
14
15
17
17
21
22
18
15
17
18
15
23
18
16
19
17
19
13
16
14
16
18
18
17
18
17
24
19
18
20
19
20
20
25
21
21
22
19
19
21
19
19
22
20
20
20
21
22
21
22
23
23
22
21
23
24
25
23
20
22
20
21
24
22
25
24
26
26
26
23
25
27
23
28
26
21
27
23
24
25
27
15
22
28
16
23
26
24
24
24
27
25
25
29
29
28
25
29
28
30
30
26
27
31
30
27
28
17
29
29
26
32
26
28
27
31
33
27
29
30
34
35
24
28
18
36
28
30
29
25
19
30
31
32
26
31
30
31
31
32
32
32
33
29
33
34
20
35
33
36
27
37
33
28
21
34
34
30
35
29
36
31
30
31
22
32
32
33
38
34
34
35
39
37
36
35
31
32
37
23
36
40
38
33
24
35
25
34
33
41
36
39
40
32
37
38
37
26
37
35
39

426
448
448
449
477
427
428
450
369
370
449
450
403
404
451
371
449
450
478
452
479
405
406
372
373
480
451
463
464
453
407
465
481
374
408
482
452
437
483
409
438
375
439
484
440
410
453
441
442
451
454
443
444
455
456
429
466
467
468
451
452
430
457
431
432
485
458
469
454
459
453
460
433
470
455
454
434
435
471
436
472
455
452
453
473
486
461
456
487
454
462
376
457
456
411
463
437
474
438
475
377
458
459
439
460
476
440
412
441
457
458
488
378
442
443
445
477
461
464
462
478
489
379
444
446
447
380
490
381
479
480
445
459
382
460
465
448
491
481
446
482
483
492
493
449
463
494
383
466
461
495
496
447
464
450
451
497
448
498
465
462
466
463
467
384
484
449
499
464
450
465
468
485
385
467
466
486
413
452
500
469
453
386
454
468
487
455
451
470
414
501
467
387
471
415
469
472
468
388
502
389
503
452
456
416
488
470
504
505
506
469
473
474
471
489
457
472
475
470
458
476
473
477
474
490
459
478
471
460
461
479
480
462
491
481
492
472
473
463
474
493
482
475
464
476
417
494
495
453
477


In [48]:
OPT_INDEX

'/workspace/DATASET/server9/STD_SRE_EGS'

In [49]:
with open(OPT_INDEX+'/spk2utt_train_len_TMP', 'wb') as handle:
    pickle.dump(spk2utt_train_len_dict_m, handle)

In [41]:
spk2utt_train_len_dict = {}
for this_dict in spk2utt_train_len_dict_m:
    for count, i in enumerate(this_dict):
        spk2utt_train_len_dict[i] = this_dict[i]
#         print(count)
print('train_len_num', len(spk2utt_train_len_dict))

train_len_num 91036


In [42]:
err_list = list(eer_m)

In [43]:
total_len = 0
for i in range(10):
    total_len += len(spk2utt_train_len_dict_m[i])
print('total', total_len + len(err_list))
print('effect', total_len)
print('err', len(err_list))

total 91061
effect 91036
err 25


In [44]:
err_list = [i[1] for i in err_list]

In [45]:
spk2utt_train_clean = {}
for i in spk2utt_train:
    spk2utt_train_clean[i] = []
    for utt in spk2utt_train[i]:
        if utt not in err_list:
            spk2utt_train_clean[i].append(utt)

In [46]:
total_len = 0
length_list = []
for i in spk2utt_train_clean:
    total_len += len(spk2utt_train_clean[i])
    length_list.append(len(spk2utt_train_clean[i]))
print('total', total_len)
print('min len', min(length_list))

total 91036
min len 1


In [47]:
with open(spk2utt_train_dict+'_clean', 'wb') as handle:
    pickle.dump(spk2utt_train_clean, handle)
with open(spk2utt_train_len, 'wb') as handle:
    pickle.dump(spk2utt_train_len_dict, handle)

In [ ]:
# spk2utt_train_len_dict = {}

In [ ]:
# err = []
# with open(spk2utt_train_dict, 'rb') as handle:
#     spk2utt_train = pickle.load(handle)

# for i in spk2utt_train:
#     for utt in spk2utt_train[i]:
#         try:
#             spk2utt_train_len_dict[utt] = librosa.core.load(utt, sr=16000)[0].shape[0]
#         except Exception:
#             err.append(utt)
#             continue
#     print(i)
            
# with open(spk2utt_train_len, 'wb') as handle:
#     pickle.dump(spk2utt_train_len_dict, handle)

In [50]:
spk2utt_val_len_dict = {}

In [51]:
err = []
with open(spk2utt_val_dict, 'rb') as handle:
    spk2utt_val = pickle.load(handle)

for i in spk2utt_val:
    for utt in spk2utt_val[i]:
        try:
            spk2utt_val_len_dict[utt] = librosa.core.load(utt, sr=16000)[0].shape[0]
        except Exception:
            err.append(utt)
            continue
    print(i)

3376
1030
6971
1090
455
1700
3084
5827
1997
6235
5242
451
3420
2704
5823
2138
5320
2267
1970
6999
4447
5294
6941
6662
3515
2896
3617
6432
5028
6174
5219
2766
1610
3747
4162
6250
1343
262
818
356
4004
2553
4785
1363
4818
79
608
6471
5422
5631
2400
5300
753
7
4605
5916
5103
1130
5319
5570
4213
5370
6281
2466
4082
4269
2069
1682
724
1297
3762
4068
5646
1285
6635
1270
4966
1525
962
5360
4386
1040
83
3744
1529
4833
354
4561
2410
2872
1195
1745
3247
1447
5853
5079
4637
1128
2518
1199
1727
808
2110
4189
2475
4188
6979
6651
5802
4645
2090
2954
4481
2781
6732
5174
3045
2160
338
6042
1542
2428
5173
6827
5852
1322
5741
234
708
5155
6722
2147
6679
1396
4141
3777
3302
788
787
3343
6775
1365
4095
4237
4124
2651
6805
3120
1348
5009
782
2950
2984
2615
2659
6034
336
2156
2096
488
5163
1937
2738
5317
1944
1180
1945
2701
1076
4784
2520
1239
4446
6904
6976
3164
3004
1210
419
1053
1784
2529
442
2731
3922
6435
3174
3705
6247
2481
1099
2287
1715
3036
3963
1186
6198
1959
5658
3468


In [52]:
err

[]

In [53]:
with open(spk2utt_val_len, 'wb') as handle:
    pickle.dump(spk2utt_val_len_dict, handle)

# Make train data

In [7]:
from preprocessing.Utils.preprocessing_3type import ThreeTypes_IterableDataset

In [8]:
data_dir_dict = {}

data_dir_dict['spk2utt_train_dict'] = os.path.join(INFO_DIR, 'spk2utt_train_dict_clean')
data_dir_dict['music_dict'] = os.path.join(AUG_INFO_DIR, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(AUG_INFO_DIR, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(AUG_INFO_DIR, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(AUG_INFO_DIR, 'rir_dict')

In [9]:
data_len_dict = {}

data_len_dict['spk2utt_train_len'] = os.path.join(INFO_DIR, 'spk2utt_train_len')
data_len_dict['music_len'] = os.path.join(AUG_INFO_DIR, 'music_len')
data_len_dict['noise_len'] = os.path.join(AUG_INFO_DIR, 'noise_len')
data_len_dict['babble_len'] = os.path.join(AUG_INFO_DIR, 'babble_len')

## Multi episode process

In [10]:
PROCESS = 1500
RUNNING_P = 10
# RUNNING_P = 1

MFCC_O = os.path.join(OPT_INDEX, 'MFCC')
LOGMELFB_O = os.path.join(OPT_INDEX, 'FB')
RAWWAV_O = os.path.join(OPT_INDEX, 'RW')
EER_LOG = os.path.join(OPT_INDEX, 'EER_LOG')
G_LOG = os.path.join(OPT_INDEX, 'G_LOG')

In [11]:
if not os.path.exists(MFCC_O):
    os.mkdir(MFCC_O)
else:
    shutil.rmtree(MFCC_O)
    os.mkdir(MFCC_O)
    
if not os.path.exists(LOGMELFB_O):
    os.mkdir(LOGMELFB_O)
else:
    shutil.rmtree(LOGMELFB_O)
    os.mkdir(LOGMELFB_O)
    
if not os.path.exists(RAWWAV_O):
    os.mkdir(RAWWAV_O)
else:
    shutil.rmtree(RAWWAV_O)
    os.mkdir(RAWWAV_O)

with open(EER_LOG, 'w') as f:
    pass

with open(G_LOG, 'w') as f:
    pass

In [12]:
config = {}

config['sr'] = 16000
config['repeats'] = 1
config['batch_size'] = 1

In [ ]:
def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (rw, fb, mfcc, label) in enumerate(dataset):                
            file_name = str(i)+str('_')+str(count)
            with open(os.path.join(RAWWAV_O, file_name), 'wb') as handle:
                pickle.dump((rw.astype(np.float16), label.astype(np.int16)), handle)
                
            with open(os.path.join(LOGMELFB_O, file_name), 'wb') as handle:
                pickle.dump((fb.astype(np.float16), label.astype(np.int16)), handle)
                
            with open(os.path.join(MFCC_O, file_name), 'wb') as handle:
                pickle.dump((mfcc.astype(np.float16), label.astype(np.int16)), handle)
            
            with open(G_LOG, 'a') as f:
                f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
            
    except Exception:
        with open(EER_LOG, 'a') as f:
            traceback.print_exc(file=f)         
        raise Exception

dataset = ThreeTypes_IterableDataset(config, data_dir_dict, data_len_dict)
dataset.noise_data_preload()

p_list = []
for i in range(PROCESS // RUNNING_P):
    p_list.append([j for j in range(i*RUNNING_P, i*RUNNING_P+RUNNING_P)])
if (PROCESS % RUNNING_P) != 0:
    p_list.append([j for j in range((i+1)*RUNNING_P, PROCESS)])

print('episode', len(p_list))
print('last episode', p_list[-1])

start_time = time.time()
for ps in p_list:
    print(ps[0])
    processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in ps]
    [p.start() for p in processes]
    joined = [p.join() for p in processes]

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
episode 150
last episode [1490, 1491, 1492, 1493, 1494, 1495, 1496, 1497, 1498, 1499]
0


In [ ]:
# # normal
# config = {}

# config['sr'] = 16000
# config['repeats'] = 150
# config['batch_size'] = 1
# config['extended_prefectch'] = 1.0
# config['running_p'] = 10
# config['out_dir'] = '/Lun0/zhiyong/dataset/test_1/'
# config['err_log'] = '/Lun0/zhiyong/dataset/tmp_data_errlog'

# def iter_data_preload(dataset, i):
#     dataset.get_random_list()
#     try:
#         start_time = time.time()
#         for count, (data, label) in enumerate(dataset):
#             with open(config['out_dir']+str(i)+str('_')+str(count), 'wb') as handle:
#                 pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
# #             print(time.time()-start_time)
#             with open(config['err_log'], 'a') as f:
#                 f.write(str(time.time()-start_time)+'\n')
#             start_time = time.time()
            
#             if ((count+1)%10000) == 0:
#                 print((count+1)//10000)
#     except Exception:
#         with open(config['err_log'], 'a') as f:
#             traceback.print_exc(file=f)
#         raise Exception

# dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# # dataset.noise_data_preload2mem()
# dataset.noise_data_preload()

# processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(config['running_p'])]
# start_time = time.time()
# [p.start() for p in processes]
# joined = [p.join() for p in processes]
# print(time.time()-start_time)

# Make train index

In [ ]:
7323*15000

In [ ]:
OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'train_data_sparse_fixed')
# expected_len = 85810
expected_len = 10984500
workers = 1500
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'train_data_sparse_fixed.csv')

In [ ]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [ ]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [ ]:
count

# Make validation data (Need only once)

In [ ]:
OPT_INDEX = '/Lun0/zhiyong/dataset'
data_dir_dict = {}

# val
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_val_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX, 'rir_dict')

In [ ]:
data_len_dict = {}

data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')
data_len_dict['music_len'] = os.path.join(OPT_INDEX, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX, 'babble_len')

In [ ]:
config = {}

config['sr'] = 16000
config['repeats'] = 5
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/dataset/val_data_1000/'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    start_time = time.time()
    for count, (data, label) in enumerate(dataset):
        with open(config['out_dir']+str(i)+'_'+str(count), 'wb') as handle:
            pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
        print(time.time()-start_time)
        start_time = time.time()

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

# Make validation index (Need only once)

In [ ]:
OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'val_data_1000')
expected_len = 1000
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'val_data_1000_csv.csv')

In [ ]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [ ]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [ ]:
count